In [1]:

from mhr.utils.utils import load_json_file, process_jsonl,write_jsonl
from py2neo import *
from neo4j import GraphDatabase
from tqdm import tqdm


mme_results_file="/mnt/petrelfs/songmingyang/code/tools/lmms-eval/scripts/logs/llava/model_args_ca10f3/0316_0009_mme_llava_baseline/mme_mul_en.json"
mme_object_file="/mnt/petrelfs/songmingyang/songmingyang/data/llava_train/benchmarks/analysis/mme/mme_llama_objects.jsonl"
mme_output_file="/mnt/petrelfs/songmingyang/songmingyang/data/llava_train/benchmarks/analysis/mme/mme_object_rank.jsonl"

In [2]:
def get_results_from_result_file(file):
    if isinstance(file, str):
        result = load_json_file(file)["logs"]
    elif isinstance(file, list):
        result = []
        for f in file:
            result.extend(load_json_file(f)["logs"])
    else:
        raise ValueError("file should be a string or a list of string")
    return result



def from_str_to_list(string):
    return string.strip().replace("[", "").replace("]", "").split(",")

def get_object_dict(object_file):
    object_data = process_jsonl(object_file)
    object_dict = {}
    for obj in object_data:
        object_dict[obj["id"]] = list(set(from_str_to_list(obj["objects"])))
    return object_dict

def get_object_statistic_by_neo4j(database="instructmix665kllama"):
    uri = "neo4j://10.140.54.16:7687"
    driver = GraphDatabase.driver(uri, auth=("neo4j", "12345678"))
    query = "match (n) return n;"
    with driver.session(database=database) as session:
        results = session.run(query).data()
        results.sort(key=lambda x: x['n']['count'], reverse=True)
        
    object_statistic_dict = {}
    for item in results:
        object_statistic_dict[item['n']['lemma']] = dict(num=item['n']['count'],rank=len(object_statistic_dict)+1)
    return object_statistic_dict 

In [3]:
object_dict = get_object_dict(mme_object_file)
print(object_dict)
object_statistic_dict = get_object_statistic_by_neo4j("instructmix665kllama")
result_list = get_results_from_result_file(mme_results_file)
analyze_results = []

wrong_max_ranks = []
correct_max_ranks = []
wrong_mean_ranks = []
correct_mean_ranks = []
wrong_min_ranks = []
correct_min_ranks = []

wrong_max_counts = []
correct_max_counts = []
wrong_mean_counts = []
correct_mean_counts = []
wrong_min_counts = []
correct_min_counts = []

for log in tqdm(result_list):
    id = log["doc"]["question_id"]
    image_key = log["doc"]["image"]
   
    score = log.get("mme_percetion_score", log.get("mme_cognition_score", None))
    if score is None:
        raise ValueError("score is None")
    score = score["score"]
    correct = score > 0.0
    
    now_object_list = object_dict.get(image_key,None)
    if now_object_list is None:
        continue
    current_num = []
    current_rank = []
    for object in now_object_list:
        if object_statistic_dict.get(object, None) is None:
            continue
        current_num.append(object_statistic_dict[object]["num"])
        current_rank.append(object_statistic_dict[object]["rank"])
    max_rank = max(current_rank)
    min_rank = min(current_rank)
    max_num = max(current_num)
    min_num = min(current_num)
    mean_rank = sum(current_rank) / len(current_rank)
    mean_num = sum(current_num) / len(current_num)
    if correct:
        correct_max_ranks.append(max_rank)
        correct_min_ranks.append(min_rank)
        correct_mean_ranks.append(mean_rank)
        correct_max_counts.append(max_num)
        correct_min_counts.append(min_num)
        correct_mean_counts.append(mean_num)
    else:
        wrong_max_ranks.append(max_rank)
        wrong_min_ranks.append(min_rank)
        wrong_mean_ranks.append(mean_rank)
        wrong_max_counts.append(max_num)
        wrong_min_counts.append(min_num)
        wrong_mean_counts.append(mean_num)
    
    
    analyze_results.append(dict(id=id, correct=correct, 
                                max_rank=max_rank, min_rank=min_rank, mean_rank=mean_rank, max_num=max_num, min_num=min_num, mean_num=mean_num,
                                rank_list=current_rank, num_list=current_num))
print("correct max rank:", sum(correct_max_ranks) / len(correct_max_ranks))
print("correct min rank:", sum(correct_min_ranks) / len(correct_min_ranks))
print("correct mean rank:", sum(correct_mean_ranks) / len(correct_mean_ranks))
print("wrong max rank:", sum(wrong_max_ranks) / len(wrong_max_ranks))
print("wrong min rank:", sum(wrong_min_ranks) / len(wrong_min_ranks))
print("wrong mean rank:", sum(wrong_mean_ranks) / len(wrong_mean_ranks))
print("correct max count:", sum(correct_max_counts) / len(correct_max_counts))
print("correct min count:", sum(correct_min_counts) / len(correct_min_counts))
print("correct mean count:", sum(correct_mean_counts) / len(correct_mean_counts))
print("wrong max count:", sum(wrong_max_counts) / len(wrong_max_counts))
print("wrong min count:", sum(wrong_min_counts) / len(wrong_min_counts))
print("wrong mean count:", sum(wrong_mean_counts) / len(wrong_mean_counts))
write_jsonl(analyze_results,mme_output_file)
        



{'numerical_calculation/images/0002.png': ['paper', 'math problem', 'answer'], 'numerical_calculation/images/0013.png': ['numbers', 'square', 'paper', 'triangle', 'diagram'], 'numerical_calculation/images/0017.png': ['pencil', 'equation', 'paper', 'table', 'pen'], 'numerical_calculation/images/0003.png': ['paper', 'equation', 'ink', 'notebook'], 'numerical_calculation/images/0011.png': ['paper', 'triangle', 'numbers', 'sections'], 'numerical_calculation/images/0016.png': ['paper', 'equation', 'ink', 'notebook'], 'numerical_calculation/images/0019.png': ['numbers', 'equation', 'ink', 'paper', 'notebook'], 'numerical_calculation/images/0008.png': ['paper', 'problem', 'numbers'], 'numerical_calculation/images/0004.png': ['', 'numbers', 'equation', 'paper', 'notebook', 'page'], 'numerical_calculation/images/0012.png': ['paper', 'number'], 'numerical_calculation/images/0001.png': ['paper', 'equation', 'ink', 'notebook'], 'numerical_calculation/images/0006.png': ['numbers', 'square', 'paper'

100%|██████████| 2372/2372 [00:00<00:00, 140467.46it/s]

correct max rank: 2708.8832976445397
correct min rank: 60.97644539614561
correct mean rank: 842.4908165086151
wrong max rank: 3222.5396825396824
wrong min rank: 131.5952380952381
wrong mean rank: 1035.4509117535906
correct max count: 32264.58137044968
correct min count: 680.5985010706638
correct mean count: 10185.18391455084
wrong max count: 33609.35317460317
wrong min count: 708.75
wrong mean count: 10457.248696145129
